In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

'''
right,so this is the latest version of our code. create a frontend for it that has the features in the project like:
1)field for user to put the utube link
2)shows the video on the screen through youtube
3)allow to userto get information from wikipedia and display it
4)allow a chat to conducted with the chatbot aboyt the information and the videos content
and so on
add buttons thatdisplay and hide these features when required and a decent ui
'''


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
!pip install langchain --quiet
!pip install langchain_community --quiet
!pip install langchain-groq --quiet
!pip install sentence-transformers --quiet
!pip install faiss-gpu --quiet
%pip install beautifulsoup4 langchain openai pandas seaborn scikit-learn langchain-community transformers sentence-transformers --quiet
!pip install -qU langchain-huggingface --quiet
!pip install datasets --quiet
%pip install -qU wikipedia --quiet
!pip install langchain_anthropic --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install youtube-transcript-api --quiet

In [6]:
import pandas as pd

from datasets import load_dataset
from google.colab import userdata
import getpass
import os
from langsmith import Client

from langchain.chains.question_answering import load_qa_chain
from langchain.schema import Document, HumanMessage, AIMessage  # Import Document class

from langchain.llms import HuggingFaceHub
from langchain_core.prompts.prompt import PromptTemplate
from langsmith.evaluation import LangChainStringEvaluator

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS

from langchain.docstore.document import Document

from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from sentence_transformers import SentenceTransformer

from itertools import chain

from langchain_community.document_loaders import WikipediaLoader
from langchain.prompts.chat import MessagesPlaceholder

# Load Data From Youtube Captions

In [7]:
import pandas as pd
from youtube_transcript_api import YouTubeTranscriptApi

def get_transcript(youtube_url, chunk_duration=10):
    try:
        # Extract Video ID from URL
        video_id = youtube_url.split("v=")[-1].split("&")[0]
        
        # Fetch English transcript
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])

        # Convert to DataFrame
        df = pd.DataFrame(transcript)

        # Grouping captions into chunks of 'chunk_duration' seconds
        grouped_transcript = []
        current_chunk = {"start": 0, "text": ""}
        
        for _, row in df.iterrows():
            if row["start"] - current_chunk["start"] < chunk_duration:
                current_chunk["text"] += " " + row["text"]
            else:
                current_chunk["duration"] = chunk_duration
                grouped_transcript.append(current_chunk)
                current_chunk = {"start": row["start"], "text": row["text"]}

        # Add last chunk
        if current_chunk:
            current_chunk["duration"] = chunk_duration
            grouped_transcript.append(current_chunk)

        # Create new DataFrame
        new_df = pd.DataFrame(grouped_transcript)

        # Save as CSV
        new_df.to_csv("youtube_transcript_grouped.csv", index=False, encoding="utf-8")
        print("✅ Transcript saved as youtube_transcript_grouped.csv")

        return new_df

    except Exception as e:
        print("❌ Error:", str(e))
        return None

# Example Usage
youtube_url = input("Enter YouTube URL: ")
df = get_transcript(youtube_url, chunk_duration=50)

# Display first few rows
if df is not None:
    print(df.head())
#https://www.youtube.com/watch?v=bXzXLahZ8Lk

Enter YouTube URL:  https://www.youtube.com/watch?v=bXzXLahZ8Lk


✅ Transcript saved as youtube_transcript_grouped.csv
     start                                               text  duration
0    0.000   Controversy erupts this morning over a planne...        50
1   50.917  on this administration's desire to try and pur...        50
2  101.668  That's in part because, the United States real...        50
3  152.619  that he I think we're going to get it one way ...        50
4  203.136  and we've heard this now repeatedly in our con...        50


# Prepare the dataset

In [8]:
texts = df['text']


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, #400,
    chunk_overlap=20
)

# Split documents into chunks
split_docs = []

for text in texts:
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        split_docs.append({"text": chunk})

print(f"Total number of chunks: {len(split_docs)}")


Total number of chunks: 12


In [45]:
split_docs[0]['text']

"Controversy erupts this morning over a planned U.S. visit to Greenland this week. It comes as President Trump has pushed to annex the autonomous Danish territory, saying he thinks the U.S. will get it one way or the other. Second Lady Bush Events and national security adviser Mike Wallace are expected to be among the Americans visiting the white House, as Lucia Vance will attend the island's national dog sled race and, quote, celebrate Greenlandic culture and unity. But Greenland's prime minister is slamming the visit, calling it, quote, highly aggressive in seeking relations to the U.S. to a new low. Let's get right to the white House. CNN. Elaina Treen is there. It's the second international trip by the second lady, Elena, and it's an odd one. But the next step in this. it is, and I think it just is really putting a further spotlight"

# Load Model

In [10]:
os.environ["GROQ_API_KEY"] = "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"
llm = ChatGroq(model="llama3-8b-8192",#"llama-3.1-8b-instant",#"llama3-8b-8192", if other one runs out
               temperature=0.7,
               verbose=True,
               timeout=None,
)

# Generate q/a pairs

In [11]:
df = pd.DataFrame(split_docs)
df=df[:25]#100

# Load QA Chain
qa_chain = load_qa_chain(llm, chain_type="stuff")  # `chain_type` can vary based on your needs

# Function to generate QA pairs
def generate_qa_pairs(row):
    try:
        # Convert the text chunk to a Document object
        doc = Document(page_content=row['text'])
        # Run the chain with the Document
        output = qa_chain.run(input_documents=[doc], question="Generate 1 question-answer pair based on the context.The question and answer should be thought-provoking in terms of the context but the answer shouldnt be very long.Your output should be like this'Question:xxxx,Answer:xxxx'.")
        return output
    except Exception as e:
        print(f"Error for chunk {row['chunk_index']}: {e}")  # Log errors
        #return None

# Apply the function to the DataFrame
df['qa_pairs'] = df.apply(generate_qa_pairs, axis=1)

# Print the resulting DataFrame
print(df[['qa_pairs']])


<ipython-input-11-9a4e84f483e1>:5: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/docs/how_to/#qa-with-rag
  qa_chain = load_qa_chain(llm, chain_type="stuff")  # `chain_type` can vary based on your needs
<ipython-input-11-9a4e84f483e1>:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = qa_chain.run(input_documents=[doc], question="Generate 1 question-answer pair based on 

                                             qa_pairs
0   Question: What might be the motivation behind ...
1   Question: Are the economic benefits of purchas...
2   Question: Is the United States' interest in pu...
3   Question: Are the escalating tensions between ...
4   Here is a question-answer pair based on the co...
5   Question: What is the true intention behind th...
6   Question: What might be the implications of Tr...
7   Question: How might Senator John Curtis' state...
8   Question: Can the White House's stance on soci...
9   Question: Does the President's return to power...
10  Question: Will Mick Mulvaney's words on cuttin...
11  Question: What inspired Alex to write a book?,...


In [12]:
#pd.set_option('display.max_colwidth', None) #for ..... error

In [13]:
x=(df[['qa_pairs']].iloc[0].to_string(index=False))
print(x)

Question: What might be the motivation behind the US government's visit to Greenland, given the controversy surrounding President Trump's comments on annexing the territory?\nAnswer: The visit may be an attempt to strengthen diplomatic ties and demonstrate the US's commitment to Greenland's independence, despite President Trump's previous comments suggesting otherwise.


# Convert to useable form for Langchain

In [14]:
import re

# Process QA pairs to create inputs and outputs lists
inputs = []
outputs = []

for qa in df["qa_pairs"]:
    try:
        # Regular expression pattern to extract question and answer
        match = re.search(r"Question:\s*(.*?)\s*Answer:\s*(.*)", qa, re.DOTALL)
        if match:
            question = match.group(1).strip()
            answer = match.group(2).strip()
            inputs.append({"question": question})
            outputs.append({"answer": answer})
        else:
            # Handle alternative format (Q: and A:)
            match_alt = re.search(r"Q:\s*(.*?)\s*A:\s*(.*)", qa, re.DOTALL)
            if match_alt:
                question = match_alt.group(1).strip()
                answer = match_alt.group(2).strip()
                inputs.append({"question": question})
                outputs.append({"answer": answer})
            else:
                print(f"Skipping malformed entry: {qa}")

    except Exception as e:
        print(f"Error processing entry: {qa}, Error: {e}")

# Display the results
print("Inputs:", inputs)
print("Outputs:", outputs)


Inputs: [{'question': "What might be the motivation behind the US government's visit to Greenland, given the controversy surrounding President Trump's comments on annexing the territory?"}, {'question': 'Are the economic benefits of purchasing Greenland worth the potential strain on its national security, as argued by the Trump administration?'}, {'question': "Is the United States' interest in purchasing Greenland primarily driven by national security concerns or economic interests?,"}, {'question': 'Are the escalating tensions between the US and Denmark a sign of a larger shift in global power dynamics?,'}, {'question': "Is it possible for President Trump to achieve a quick deal with Russia without sacrificing Ukraine's interests, given his eagerness to do so?"}, {'question': "What is the true intention behind the US National Security Adviser's trip to Greenland?,"}, {'question': "What might be the implications of Trump's annexation efforts on international relationships, particularly

# Langsmith Setup

In [15]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_API_KEY"] = "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"
#os.environ["LANGCHAIN_API_KEY"] = userdata.get("lsmith")

os.environ["LANGCHAIN_ENDPOINT"]= "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="default"

In [16]:
'''
To hcheck project
runs = list(client.list_runs(project_name="default"))
if runs:
    url = runs[0].url
    print(url)
else:
    print("No runs found in the 'default' project.")
'''

'\nTo hcheck project\nruns = list(client.list_runs(project_name="default"))\nif runs:\n    url = runs[0].url\n    print(url)\nelse:\n    print("No runs found in the \'default\' project.")\n'

**To check if Langsmith is working**

In [20]:
'''
client = Client()

url = next(client.list_runs(project_name="default")).url
print(url)
'''

'\nclient = Client()\n\nurl = next(client.list_runs(project_name="default")).url\nprint(url)\n'

In [21]:
client = Client()

# Define dataset: these are your test cases
dataset_name = "FYP3 Example Dataset"

In [22]:
# Define the evaluation prompt template
_PROMPT_TEMPLATE = """You are an expert professor specialized in grading students' answers to questions.You have to grade the answer as CORRECT oe INCORRECT
based on the conceptual accuracy of the answer.Grammatical mistakes are fine, as long as the answer is close enough to the real answer
You are grading the following question:
{query}
Here is the real answer:
{answer}
You are grading the following predicted answer:
{result}
Respond with CORRECT or INCORRECT:
Grade:
"""

PROMPT = PromptTemplate(
    input_variables=["query", "answer", "result"], template=_PROMPT_TEMPLATE
)

# Use Hugging Face model
eval_llm = llm

qa_evaluator = LangChainStringEvaluator("qa", config={"llm": eval_llm, "prompt": PROMPT})


In [23]:
from langsmith.schemas import Run, Example

def evaluate_length(run: Run, example: Example) -> dict:
    prediction = run.outputs.get("output")
    required = example.outputs.get("answer")
    score = int(len(prediction) < 2 * len(required))
    return {"key":"length", "score": score}

def my_app(question):
    print(question)
    user_input = input("Please enter a string: ")
    return user_input
def langsmith_app(inputs):
    output = my_app(inputs["question"])
    return {"output": output}


In [25]:
'''
#to check all datasets
from langsmith import Client
client = Client()
datasets = list(client.list_datasets())  # Convert generator to list
print(datasets)  # This will show the actual dataset names

# Check if dataset exists
datasets = [d.name for d in client.list_datasets()]
if dataset_name not in datasets:
    client.create_dataset(dataset_name)
    print(f"Dataset '{dataset_name}' created successfully!")
else:
    print(f"Dataset '{dataset_name}' already exists.")
'''

from langsmith import Client

client = Client()

# Define dataset name
dataset_name = "FYP3 Draft Dataset"
datasets = list(client.list_datasets())  # Convert generator to list

if dataset_name not in datasets:
    client.create_dataset(dataset_name)
    print(f"Dataset '{dataset_name}' created successfully!")
else:
    print(f"Dataset '{dataset_name}' already exists.")

# Retrieve dataset and get its UUID
datasets = {d.name: d for d in client.list_datasets()}
if dataset_name not in datasets:
    raise ValueError(f"Dataset '{dataset_name}' not found.")

dataset_id = str(datasets[dataset_name].id)  # Ensure we use the UUID

# Add examples to the dataset
for input_data, output_data in zip(inputs, outputs):
    client.create_example(
        dataset_id=dataset_id,
        inputs={"question": input_data["question"]},
        outputs={"answer": output_data["answer"]}
    )

print(f"Successfully added {len(inputs)} examples to dataset '{dataset_name}'.")


Dataset 'FYP3 Draft Dataset' created successfully!
Successfully added 12 examples to dataset 'FYP3 Draft Dataset'.


# Evaluate q/a with user reponse

In [26]:
sample_user_answers = [
    {"answer": "I think both are important. Small discoveries add up, but sometimes big changes in ideas are needed too."},
    {"answer": "Kuhn's paradigm is like a framework. Scientists follow it until new evidence shows it is wrong."},
    {"answer": "Scientists keep learning new things and fixing their ideas when problems or new stuff appear."},
    {"answer": "Sometimes, strong views can block new ideas, but smart people always find ways to improve."},
    {"answer": "Yes, Fresnel's work changed how we see light, using math to explain waves better."},
    {"answer": "Polarization showed how new ideas can challenge old ones and make big changes in science."},
    {"answer": "Both lone geniuses and teamwork are important, but teamwork helps share and grow ideas faster."},
    {"answer": "Yes, many scientists used Galileo's ideas to make their own work better."},
    {"answer": "Huygens is important because his work helped others, even if we don't remember him much."},
    {"answer": "No, science is teamwork. Even Newton and Galileo built on others' work."},
    {"answer": "We are maybe early still. Science has so much more to discover."}
]
answers = [d['answer'] for d in sample_user_answers]
print(answers)

['I think both are important. Small discoveries add up, but sometimes big changes in ideas are needed too.', "Kuhn's paradigm is like a framework. Scientists follow it until new evidence shows it is wrong.", 'Scientists keep learning new things and fixing their ideas when problems or new stuff appear.', 'Sometimes, strong views can block new ideas, but smart people always find ways to improve.', "Yes, Fresnel's work changed how we see light, using math to explain waves better.", 'Polarization showed how new ideas can challenge old ones and make big changes in science.', 'Both lone geniuses and teamwork are important, but teamwork helps share and grow ideas faster.', "Yes, many scientists used Galileo's ideas to make their own work better.", "Huygens is important because his work helped others, even if we don't remember him much.", "No, science is teamwork. Even Newton and Galileo built on others' work.", 'We are maybe early still. Science has so much more to discover.']


In [27]:
from langsmith import evaluate

experiment_results = evaluate(
    langsmith_app, # Your AI system
    data=dataset_name, # The data to predict and grade over
    evaluators=[evaluate_length, qa_evaluator], # The evaluators to score the results
    experiment_prefix="fyp3_q_a_evaluation", # A prefix for your experiment names to easily identify them
)

View the evaluation results for experiment: 'fyp3_q_a_evaluation-1ecdcd64' at:
https://smith.langchain.com/o/2a3a5340-52b0-4dbd-904f-a36931fa8e07/datasets/5a43c725-a3de-449a-920e-37d0a27e0424/compare?selectedSessions=6ce2f85c-b5cc-49b6-b801-85e95d13ee8d




0it [00:00, ?it/s]

Will Mick Mulvaney's words on cutting entitlements have any significant impact on the political landscape?


Please enter a string:  We are maybe early still. Science has so much more to discover.


Does the President's return to power pose a threat to the integrity of Social Security benefits, as claimed by the Commerce secretary?


...

Please enter a string:  We are maybe early still. Science has so much more to discover.


Are the economic benefits of purchasing Greenland worth the potential strain on its national security, as argued by the Trump administration?


Please enter a string:  We are maybe early still. Science has so much more to discover.


What might be the motivation behind the US government's visit to Greenland, given the controversy surrounding President Trump's comments on annexing the territory?


Please enter a string:  We are maybe early still. Science has so much more to discover.


# Embed and make vectorstore

In [28]:
# Convert dictionaries to Document objects
initial_docs = [Document(page_content=doc["text"], metadata={}) for doc in split_docs]

# Create embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create FAISS vector store from Document objects
vectorStore = FAISS.from_documents(initial_docs, embedding=embedding_model)

print("Vector store created successfully!")

<ipython-input-28-0c855150ecc5>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store created successfully!


In [29]:
def create_vector_store(docs):
    embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    vectorStore = FAISS.from_documents(docs, embedding=embedding)

    return vectorStore

# Initialize documents for vectorstore

In [51]:
# Convert `ds["text"]` entries to Document objects
initial_docs = [
    Document(metadata={}, page_content=text) for text in df["text"][:1].tolist()
]

# Create the vector store with the combined documents
vectorStore = create_vector_store(initial_docs)

# Get wikipedia page of query

In [32]:
def process_query_to_wikipedia(query):
  data = WikipediaLoader(query=query, load_max_docs=2).load()
  link=data[0].metadata["source"] # metadata of the first document
  #data[0].page_content[:400]
  return link

# Functions to get data from link and create chain from Vectorstore

In [33]:
# Retrieve Data
def get_docs_from_wikipedia(link):
    loader = WebBaseLoader(link)
    docs = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=20
    )
    splitDocs = text_splitter.split_documents(docs)

    return splitDocs

def create_chain(vectorStore):
    model = llm

    prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer the user's questions based on the context: {context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ])


    # chain = prompt | model
    document_chain = create_stuff_documents_chain(
        llm=model,
        prompt=prompt
    )

    retriever = vectorStore.as_retriever()

    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    return retrieval_chain

In [34]:
def process_chat(chain, question, chat_history):
    response = chain.invoke({
        "chat_history": chat_history,
        "input": question,
    })
    return response["answer"]

# Update vector store with new docs

In [42]:
def update_vector_store(vectorStore, link):
    docs_from_website = get_docs_from_wikipedia(link)

    # Add new embeddings and documents to the existing vector store
    vectorStore.add_documents(docs_from_website)

    return vectorStore

In [53]:

#print everything in a vectorstore
doc_ids = vectorStore.docstore._dict.keys()

for doc_id in doc_ids:
       document = vectorStore.docstore.search(doc_id)
       print(document)


page_content='Controversy erupts this morning over a planned U.S. visit to Greenland this week. It comes as President Trump has pushed to annex the autonomous Danish territory, saying he thinks the U.S. will get it one way or the other. Second Lady Bush Events and national security adviser Mike Wallace are expected to be among the Americans visiting the white House, as Lucia Vance will attend the island's national dog sled race and, quote, celebrate Greenlandic culture and unity. But Greenland's prime minister is slamming the visit, calling it, quote, highly aggressive in seeking relations to the U.S. to a new low. Let's get right to the white House. CNN. Elaina Treen is there. It's the second international trip by the second lady, Elena, and it's an odd one. But the next step in this. it is, and I think it just is really putting a further spotlight'


# Create Chain

In [36]:
# Create the chain
chain = create_chain(vectorStore)

# Chat with Model


In [43]:
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    link=process_query_to_wikipedia(user_input)
    update_vector_store(vectorStore, link)

    response = process_chat(chain, user_input, chat_history)

    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=response))
    print("Assistant:", response)

You:  what is mick mulvaney saying?


Assistant: According to the article, Mick Mulvaney is quoted as saying:

"You're not seeing a correlation between the economic numbers and Joe Biden's approval" - in a CNBC interview on February 23, 2024.

(Mick Mulvaney resigned from the Trump administration in January 2021, so this quote is likely from a post-resignation interview or article.


You:  what is mick mulvaneys experience in the political landscape


Assistant: According to the provided sources, Mick Mulvaney's experience in the political landscape includes:

* South Carolina House of Representatives: Mulvaney served in the South Carolina House of Representatives.
* U.S. Congress: Mulvaney was a U.S. Representative for South Carolina's 5th congressional district from 2011 to 2017.
* Trump Administration: Mulvaney served as:
	+ Director of the Office of Management and Budget (OMB) from 2017 to 2020.
	+ Acting White House Chief of Staff from 2019 to 2020.

Additionally, Mulvaney has been involved with various organizations and think tanks, such as:

* Heritage Action For America: Mulvaney is a member of the Heritage Action For America's Board of Directors.
* The Heritage Foundation: Mulvaney has been a fellow at the Heritage Foundation.

It's worth noting that the provided sources do not mention any other political experience or positions held by Mulvaney beyond the ones mentioned above.


KeyboardInterrupt: Interrupted by user